In [ ]:
# checking of python version

import sys
print(sys.version)

In [ ]:
# mount your google drive so that experiment logs can be saved

from google.colab import drive
drive.mount('/content/drive')

# **Julia Installation**

In [ ]:
# this notebook will require installation of Julia

%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0, julia version must be the same in order to produce reproduciable results
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2024-12-26 14:38:09 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# **Commands to get information newely installed Julia**

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [2]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  484.695 ms (2 allocations: 32.00 MiB)


In [3]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


# **Installation of required packages**

In [4]:
# installation of packages
using Pkg
Pkg.add(["XLSX", "Test", "DataFrames", "JuMP", "StatsBase", "HiGHS", "PyCall", "CSV"])
Pkg.add(PackageSpec(name = "SDDP", version = "1.5.0"))

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed IrrationalConstants ───────── v0.2.2
   Installed InputBuffers ──────────────── v1.0.0
   Installed Crayons ───────────────────── v4.1.1
   Installed DiffRules ─────────────────── v1.15.1
   Installed ZipArchives ───────────────── v2.4.0
   Installed EzXML ─────────────────────── v1.2.0
   Installed MutableArithmetics ────────── v1.6.0
   Installed XML2_jll ──────────────────── v2.13.5+0
   Installed ArgCheck ──────────────────── v2.4.0
   Installed TableTraits ───────────────── v1.0.1
   Installed PtrArrays ─────────────────── v1.2.1
   Installed DiffResults ───────────────── v1.1.0
   Installed SpecialFunctions ──────────── v2.5.0
   Installed Bzip2_jll ─────────────────── v1.0.8+3
   Installed CodecBzip2 ────────────────── v0.8.5
   Installed SentinelArrays ────────────── v1.4.8
   Installed Tables ────────────────────── v1.12.0
   Installed PyCall ────────────────────── v1.96.4

# **Model Parameters**

In [5]:
no_aa = 4            # number of affected areas
no_m = 1               # number of materials
no_time_periods = 12  # number of time periods

aaa = reshape([2.04 for i in 1 : no_aa], (no_aa, no_m))      # list of coefficients to be used in deprivation cost function. The size of the array is equal to "no_affected_areas" times "no_materials".
bbb = reshape([0.24 for i in 1 : no_aa], (no_aa, no_m))      # list of coefficients to be used in deprivation cost function. The size of the array is equal to "no_affected_areas" times "no_materials".

demand_aa_mat = 1.5 * reshape([1 for i in 1 : no_aa], (no_aa, no_m))           # robustified demand values generated at affected areas for relief item(s)

# 200, 250, 300, 350, 400, 450, 500, 550, 600, 650
cost_path = reshape([200, 250, 300, 350], (no_aa, no_m))     # number of time periods times number of affected areas (AAs) times number of materials (accessibility based delivery cost for unit capacity)

cap_dist = 3    # total flexible capacity at local response centers (LRCs). The size of this list is equal to "no_dist_centers". The order is "no_dist_centers". (1, 2, 3)

3

# **Problem definition (multi-period, location relocation, resource allocation two echelon supply chain problem)**

In [6]:
# setting random seed for reproducbility of simulation experiments

using Random, CSV, DataFrames
Random.seed!(1)

TaskLocalRNG()

In [7]:
# setting random seed for reproducbility of simulation experiments

using SDDP, HiGHS, Test, XLSX, DataFrames, JuMP, StatsBase, Random, CSV

#################################### specification of data #########################################

no_lrc = 1               # number of local response centers (6, 9, 12)
Len = 4  # Length of one time period
planning_horizon = Len * no_time_periods

logistics_cost_weight = 1 / 3
absolute_deprivation_cost_weight = 1 / 3

linear_interval = 4     # to be used for the linearization of the deprivation cost function (which is exponential)

state_init = [0 for i in 1 : no_aa * no_m]

consum_cap_dist = reshape([1], (no_m, no_lrc))    # The size of this list is equal to "no_dist_centers" times number of materials. The order is "no_dist_centers" and then number of materials (its units (number of units of relief item per capacity consumed) will be changed in case of large values of deamnd in order to avoid numerical instablility issues)

if cap_dist <= demand_aa_mat[1]
  tmep_arr = [x for x = round(maximum(demand_aa_mat) - cap_dist, digits = 1) : 0.5 : round(maximum(demand_aa_mat) * no_time_periods, digits = 1)]
else
  tmep_arr = [x for x = round((maximum(demand_aa_mat) - cap_dist) * no_time_periods, digits = 1) : 0.5 : round(maximum(demand_aa_mat) * no_time_periods, digits = 1)]
end

tmep_arr_zeros = zeros(size(tmep_arr))

for i in 1 : size(tmep_arr_zeros)[1]
  tmep_arr_zeros[i] = (tmep_arr[i] >= 0)
end

scal_fact = 1

1

In [8]:
sup_chain_model = SDDP.LinearPolicyGraph(
    stages = no_time_periods,
    lower_bound = -10.0,
    optimizer = HiGHS.Optimizer,
    # solver = IpoptSolver(print_level=0)
) do subproblem, stage

    ############################################# state variables ######################################

    # deprivation level of each affected area for each material
    @variable(subproblem, minimum((minimum(tmep_arr), 0)) <= dep_lvl_state_var[i = 1 : no_aa, j = 1 : no_m] <= maximum(tmep_arr), SDDP.State, initial_value = 0)


    ################################################## decision variables ###############################################

    # resource allocation variable definition (from local response centers to affected areas for a particular relief item)
    @variables(subproblem, begin
        0 <= res_alloc_var[i = 1 : no_aa, j = 1 : no_m, k = 1 : no_lrc] <= cap_dist, Int
    end)

    # values to be used in the definition of the first set of linearization variables
    @variables(subproblem, begin
        val_lin_var_1[i = 1 : no_aa, j = 1 : no_m, k = 1 : size(tmep_arr)[1]], Bin
    end)

    ############################################## Transition Functions ###############################################

    # deprivation level of each affected area for each material
    @constraints(subproblem, begin
      dep_lvl_trans_f[i = 1 : no_aa, j = 1 : no_m], dep_lvl_state_var[i, j].out / scal_fact == (dep_lvl_state_var[i, j].in - sum(res_alloc_var[i, j, k] for k in 1 : no_lrc) + demand_aa_mat[i, j]) / scal_fact
    end)

    ############################################### constraints ###################################################

    # local response centers' flexible capacity constraints
    @constraints(subproblem, begin
      c_1[i = 1 : no_lrc], sum(res_alloc_var[j, k, i] for k in 1 : no_m for j in 1 : no_aa) / scal_fact <= cap_dist / scal_fact
    end)

    # linearization constraints which includes state variables
    @constraints(subproblem, begin
      c_2[i = 1 : no_aa, j = 1 : no_m], dep_lvl_state_var[i, j].out / scal_fact == sum(tmep_arr[k] * val_lin_var_1[i, j, k] for k in 1 : size(tmep_arr)[1]) / scal_fact
    end)

    # linearization constraints for first set of linarization variables
    @constraints(subproblem, begin
      c_3[i = 1 : no_aa, j = 1 : no_m], sum(val_lin_var_1[i, j, k] for k in 1 : size(tmep_arr)[1]) == 1
    end)

    ############################################################# objective function ###################################################

    # accessibility based delivery cost
    @expression(
      subproblem, access_deliv_cost_exp,
      sum(cost_path[i, j, k] * res_alloc_var[i, j, k] for k in 1 : no_lrc for j in 1 : no_m for i in 1 : no_aa)
    )

    if stage == 1
      @expression(
        subproblem, dep_cost_exp,
        sum((exp(aaa[i, j]) * (exp(bbb[i, j] * linear_interval) - 1)) * (exp(bbb[i, j] * linear_interval) ^ tmep_arr[k]) * tmep_arr_zeros[k] * val_lin_var_1[i, j, k] for k in 1 : size(tmep_arr)[1] for j in 1 : no_m for i in 1 : no_aa) +
        sum((exp(aaa[i, j]) * (exp(bbb[i, j] * linear_interval) - 1)) for j in 1 : no_m for i in 1 : no_aa)
      )
    else
      @expression(
        subproblem, dep_cost_exp,
        sum((exp(aaa[i, j]) * (exp(bbb[i, j] * linear_interval) - 1)) * (exp(bbb[i, j] * linear_interval) ^ tmep_arr[k]) * tmep_arr_zeros[k] * val_lin_var_1[i, j, k] for k in 1 : size(tmep_arr)[1] for j in 1 : no_m for i in 1 : no_aa)
      )
    end

    @stageobjective(
      subproblem,
      (logistics_cost_weight * access_deliv_cost_exp + absolute_deprivation_cost_weight * dep_cost_exp)
    )
end

A policy graph with 12 nodes.
 Node indices: 1, ..., 12


In [9]:
for i in 1 : no_time_periods
  SDDP.parameterize(sup_chain_model[i], 1.1)
  SDDP.write_subproblem_to_file(sup_chain_model[i], "/content/drive/MyDrive/nested_benders_decomposition_($(no_aa),_$(cap_dist),_$(no_time_periods))_model_of_stage_$(i)_sup_chain_model.lp")
end

read("sup_chain_model.lp") |> String |> print

LoadError: SystemError: opening file "sup_chain_model.lp": No such file or directory

In [10]:
@time begin

# SDDP agent training
SDDP.train(
    sup_chain_model,
    #iteration_limit = 30,
    stopping_rules = [SDDP.BoundStalling(1000, 1e-4)],
    cut_type = SDDP.SINGLE_CUT,
    # cut_type = SDDP.MULTI_CUT,
    log_frequency = 100,
    risk_measure = SDDP.Expectation(),     # for risk neutral decision-making
    # risk_measure = SDDP.EAVaR(;lambda=0.5, beta=0.5),     # for risk-averse decision-making
)

end

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 12
  state variables : 4
  scenarios       : 1.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [305, 305]
  AffExpr in MOI.EqualTo{Float64}         : [12, 12]
  AffExpr in MOI.LessThan{Float64}        : [1, 1]
  VariableRef in MOI.GreaterThan{Float64} : [9, 9]
  VariableRef in MOI.Integer              : [4, 4]
  VariableRef in MOI.LessThan{Float64}    : [8, 9]
  VariableRef in MOI.ZeroOne              : [292, 292]
numerical stability report
  matrix range     [5e-01, 2e+01]
  objective range  [1e+00, 1e+08]
  bounds range     [3e+00, 2e+01]
  rhs range        [1e+00, 3e+00]
  - objective range

In [11]:
filePath = "/content/drive/MyDrive/nested_benders_decomposition_($(no_aa),_$(cap_dist),_$(no_time_periods))_convergence_curves.csv"
SDDP.write_log_to_csv(sup_chain_model, filePath)

CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)

CSV.write(filePath, DataFrame([["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"], ["####"]], :auto), append = true)
CSV.write(filePath, DataFrame([["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ], :auto), append = true)

"/content/drive/MyDrive/nested_benders_decomposition_(4,_3,_12)_convergence_curves.csv"

# **Evaluation of the learned policy**

In [12]:
simulations = SDDP.simulate(
  # The trained model to simulate.
  sup_chain_model,
  # The number of replications.
  1,
  # A list of names to record the values of.
  [:dep_lvl_state_var, :res_alloc_var, :val_lin_var_1],
)

1-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:val_lin_var_1 => [0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; … ;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0], :dep_lvl_state_var => SDDP.State{Float64}[SDDP.State{Float64}(0.0, 0.5000000000004903); SDDP.State{Float64}(0.0, 0.5); SDDP.State{Float64}(0.0, 1.5); SDDP.State{Float64}(0.0, 0.4999999999995097);;], :bellman_term => 186727.70348502413, :noise_term => nothing, :res_alloc_var => [0.9999999999995097; 1.0; 0.0; 1.0000000000004898;;;], :node_index => 1, :stage_objective => 320.6627689107156, :objective_state => nothing, :belief => Dict(1 => 1.0)), Dict(:val_lin_var_1 => [0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; … ;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0], :dep_lvl_state_var => SDDP.State{Float64}[SDDP.State{Float64}(0.5000000000004903, 2.0000000000004903); SDDP.State{Float64}(0.5, 2.0); SDDP.State{Float64}(1.5, 1.0); SDDP.State{Float64}

In [13]:
CSV.write(filePath, DataFrame([["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"]], :auto), append = true)
CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)

total_objective = 0
for i in 1 : no_time_periods
  println(simulations[1][i][:stage_objective])
  CSV.write(filePath, DataFrame([["Stage $i Objective Value: "], [simulations[1][i][:stage_objective]]], :auto), append = true)
end

CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)

320.6627689107156
394.6113841755208
370.7365477296805
594.4088380298449
967.4924779793848
1671.8618239437646
3011.3096166735563
5494.679467155858
12140.893348061045
25068.959182742154
49160.4859585861
93680.15276741743


"/content/drive/MyDrive/nested_benders_decomposition_(4,_3,_12)_convergence_curves.csv"

In [14]:
CSV.write(filePath, DataFrame([["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"]], :auto), append = true)
CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)
CSV.write(filePath, DataFrame([["Resource Allocation Decisions:"]], :auto), append = true)

res_alloc_dec_var_dict = Dict()     # this dictonary will store the decision (control) variables from the trained policy

for i in 1 : no_time_periods
  println(reshape(simulations[1][i][:res_alloc_var], (no_aa, no_m, no_lrc)))
  CSV.write(filePath, DataFrame([[round(simulations[1][i][:res_alloc_var][j], digits = 0)] for j in 1 : no_aa * no_m], :auto), append = true)
  res_alloc_dec_var_dict["$i"] = reshape(simulations[1][i][:res_alloc_var], (no_aa, no_m, no_lrc))
end

[0.9999999999995097; 1.0; 0.0; 1.0000000000004898;;;]
[-3.723243571003754e-16; -0.0; 2.0; 1.0;;;]
[2.0000000000000004; 1.0; 0.0; -0.0;;;]
[-0.0; 1.0; 1.0; 1.0;;;]
[-0.0; 1.0; 1.0; 1.0;;;]
[1.0000000000000004; -0.0; 1.0; 1.0;;;]
[0.9999999999999997; 1.0; -0.0; 1.0;;;]
[1.0000000000000004; 1.0; 1.0; -0.0;;;]
[0.9999999999999997; 1.0; 1.0; -0.0;;;]
[-7.84595772828248e-17; 4.440892098500626e-16; 1.0; 2.0;;;]
[1.0; 1.0; -0.0; 1.0;;;]
[1.0; 1.0; 1.0; -0.0;;;]


In [15]:
CSV.write(filePath, DataFrame([["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"]], :auto), append = true)
CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)
CSV.write(filePath, DataFrame([["Values of the Variables for the Linearization of Non-Convex Objective:"]], :auto), append = true)

val_lin_var_1_dict = Dict()     # this dictonary will store the decision (control) variables from the trained policy

for i in 1 : no_time_periods
  println(reshape(simulations[1][i][:val_lin_var_1], (no_aa, no_m, size(tmep_arr)[1])))
  CSV.write(filePath, DataFrame([[round(simulations[1][i][:val_lin_var_1][j], digits = 0)] for j in 1 : no_aa * no_m * size(tmep_arr)[1]], :auto), append = true)
  val_lin_var_1_dict["$i"] = reshape(simulations[1][i][:val_lin_var_1], (no_aa, no_m, size(tmep_arr)[1]))
end

[0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.9999999999995097; 1.0; -0.0; 1.0;;; 0.0; 0.0; 0.0; 0.0;;; 4.902744876744691e-13; -0.0; 1.0; 2.220446049250313e-16;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 0.0; 0.0; 0.0; 0.0;;; 

# **Function for evaluation of learned policy on real objective function (not linearized one)**

In [16]:
function eval_non_convex_obj_fn(res_alloc_dec_var_dict, no_lrc, no_aa, no_m, state_init, demand_aa_mat, aaa, bbb, Len,
                               cost_path, logistics_cost_weight, absolute_deprivation_cost_weight)
  # computation of state variables for all time periods
  dep_lvl_state_var_dict = Dict()
  dep_lvl_state_var_dict["0"] = state_init

  for i in 1 : no_time_periods
    state_app = zeros(no_aa, no_m)
    for j in 1 : no_aa
      for k in 1 : no_m
        state_app[j, k] = dep_lvl_state_var_dict["$(i - 1)"][j, k] - sum(res_alloc_dec_var_dict["$i"][j, k, l] for l in 1 : no_lrc) + demand_aa_mat[j, k]
      end
    end
    dep_lvl_state_var_dict["$i"] = state_app
  end

  # calculation of deprivation cost
  dep_cost_sum = 0                                            # Before calculating the total deprivation cost, the class attribute "deprivation cost" is set to "0".
  for (s_keys, state_dict_temp) in dep_lvl_state_var_dict

    counter = 0
    for i in 1 : no_aa                                               # This for loop loops through all the affected areas and calculate the total deprivation cost.
      for j in 1 : no_m
        if (state_dict_temp[i, j] >= 0)                                                                   # It the state value of the affected area is greater then or equal to "0" then the formula below is used.
          dep_cost_sum = dep_cost_sum + exp(aaa[i, j]) * (exp(bbb[i, j] * Len) - 1) * (exp(bbb[i, j] * Len) ^ state_dict_temp[i, j])                        # This formula is used to calculate the deprivation cost for a particular Affected Area, given a particular state value of that Affected Area.
        else
          dep_cost_sum = dep_cost_sum                                                                 # If the state value of the Affected Area is less then "0" then the deprivation cost of that affected area will be set to "0".
        end
      end
      counter = counter + 1
    end

  end

  # logistics cost
  log_cost = 0
  for i in 1 : no_time_periods
    for j in 1 : no_aa
      for k in 1 : no_m
        for l in 1 : no_lrc
          log_cost = log_cost + cost_path[j, k, l] * res_alloc_dec_var_dict["$i"][j, k, l]
        end
      end
    end
  end

  return dep_lvl_state_var_dict, absolute_deprivation_cost_weight * dep_cost_sum + logistics_cost_weight * log_cost
end

dep_lvl_state_var_dict, real_obj_val = eval_non_convex_obj_fn(res_alloc_dec_var_dict, no_lrc, no_aa, no_m, reshape([0 for i in 1 : no_aa * no_m], (no_aa, no_m)), demand_aa_mat, aaa, bbb, Len, cost_path, logistics_cost_weight, absolute_deprivation_cost_weight)

(Dict{Any, Any}("4" => [3.0000000000004903; 3.0; 3.0; 2.99999999999951;;], "1" => [0.5000000000004903; 0.5; 1.5; 0.49999999999951017;;], "12" => [9.00000000000049; 9.0; 9.0; 8.99999999999951;;], "2" => [2.0000000000004907; 2.0; 1.0; 0.9999999999995102;;], "6" => [5.00000000000049; 5.0; 4.0; 3.99999999999951;;], "11" => [8.50000000000049; 8.5; 8.5; 7.49999999999951;;], "5" => [4.50000000000049; 3.5; 3.5; 3.49999999999951;;], "7" => [5.50000000000049; 5.5; 5.5; 4.49999999999951;;], "8" => [6.00000000000049; 6.0; 6.0; 5.99999999999951;;], "10" => [8.00000000000049; 8.0; 7.0; 6.99999999999951;;]…), 192876.25418141068)

In [17]:
CSV.write(filePath, DataFrame([["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ["-"], ], :auto), append = true)
CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)
CSV.write(filePath, DataFrame([["Deprivation Levels of Affected Areas (State Variables):"]], :auto), append = true)

for i in 0 : no_time_periods
  println(dep_lvl_state_var_dict["$(i)"])
  CSV.write(filePath, DataFrame([[round(dep_lvl_state_var_dict["$(i)"][j], digits = 1)] for j in 1 : no_aa * no_m], :auto), append = true)
end

[0; 0; 0; 0;;]
[0.5000000000004903; 0.5; 1.5; 0.49999999999951017;;]
[2.0000000000004907; 2.0; 1.0; 0.9999999999995102;;]
[1.5000000000004903; 2.5; 2.5; 2.49999999999951;;]
[3.0000000000004903; 3.0; 3.0; 2.99999999999951;;]
[4.50000000000049; 3.5; 3.5; 3.49999999999951;;]
[5.00000000000049; 5.0; 4.0; 3.99999999999951;;]
[5.50000000000049; 5.5; 5.5; 4.49999999999951;;]
[6.00000000000049; 6.0; 6.0; 5.99999999999951;;]
[6.50000000000049; 6.5; 6.5; 7.49999999999951;;]
[8.00000000000049; 8.0; 7.0; 6.99999999999951;;]
[8.50000000000049; 8.5; 8.5; 7.49999999999951;;]
[9.00000000000049; 9.0; 9.0; 8.99999999999951;;]


In [18]:
CSV.write(filePath, DataFrame([["\n"]], :auto), append = true)
CSV.write(filePath, DataFrame([["Total Objective Value: "], [real_obj_val]], :auto), append = true)
real_obj_val

192876.25418141068

In [19]:
real_obj_val

192876.25418141068

# **Evaluation of Solution from Julia language in Python language (by running python program in Julia)**

In [ ]:
using PyCall

py"""
from math import exp
import numpy as np

def eval_non_convex_obj_fn(dep_lvl_state_var_dict, res_alloc_dec_var_dict, no_time_periods, no_lrc, no_aa, no_m, aaa, bbb, Len,
                               cost_path, logistics_cost_weight, absolute_deprivation_cost_weight):

  dep_lvl_state_var_dict = np.array(dep_lvl_state_var_dict)
  res_alloc_dec_var_dict = np.array(res_alloc_dec_var_dict)

  res_alloc_dec_var_dict = res_alloc_dec_var_dict
  aaa = np.array(aaa).reshape(no_aa, no_m)
  bbb = np.array(bbb).reshape(no_aa, no_m)
  cost_path = np.array(cost_path).reshape(no_time_periods, no_aa, no_m, no_lrc)

  # calculation of deprivation cost
  dep_cost_sum = 0                                            # Before calculating the total deprivation cost, the class attribute "deprivation cost" is set to "0".
  for state_dict_temp in dep_lvl_state_var_dict:
    for i in range(0, no_aa):                                               # This for loop loops through all the affected areas and calculate the total deprivation cost.
      for j in range(0, no_m):
        if (state_dict_temp[i, j] >= 0):                                                                   # It the state value of the affected area is greater then or equal to "0" then the formula below is used.
          dep_cost_sum = dep_cost_sum + exp(aaa[i, j]) * (exp(bbb[i, j] * Len) - 1) * (exp(bbb[i, j] * Len) ** state_dict_temp[i, j])                        # This formula is used to calculate the deprivation cost for a particular Affected Area, given a particular state value of that Affected Area.
        else:
          dep_cost_sum = dep_cost_sum                                                                 # If the state value of the Affected Area is less then "0" then the deprivation cost of that affected area will be set to "0".

  # logistics cost
  log_cost = 0
  for i in range(0, no_time_periods):
    for j in range(0, no_aa):
      for k in range(0, no_m):
        for l in range(0, no_lrc):
          log_cost = log_cost + cost_path[i, j, k, l] * res_alloc_dec_var_dict[i, j, k, l]

  return absolute_deprivation_cost_weight * dep_cost_sum + logistics_cost_weight * log_cost
"""

In [ ]:
dep_lvl_state_var_dict = collect(values(dep_lvl_state_var_dict))
res_alloc_dec_var_dict = collect(values(res_alloc_dec_var_dict))

real_obj_val = py"eval_non_convex_obj_fn"(dep_lvl_state_var_dict, res_alloc_dec_var_dict, no_time_periods, no_lrc, no_aa, no_m, aaa, bbb, Len,
                                                                    cost_path, 1 / 3, 1 / 3)

# some how it is not working as it is giving incorrect results

LoadError: ignored

In [ ]:
res_alloc_dec_var_dict

Any[]